In [1]:
import os

# environment setup
with open("./config/.key", "r") as key_file:
    keys = list(key_file)

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

In [2]:
# Image Generation Module
from ImageGeneration.edenai import *

# Text Generation Module
from TextGeneration.SimpleGeneration import *
from TextGeneration.CatelogueGeneration import *
from TextGeneration.ReferencePostGeneration import *

# Moments Module
from Moments.Moments import *

d:\ML stuff\buzztrends\API/ImageGeneration/
Creating Indexes from Handbook...
Indexes created Successfully


In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.vectorstores.base import VectorStoreRetriever
from utils.utils import get_llm

def generate_content(
    company_name: str,
    moment: str,
    content_type: str,
    tone: str,
    objective: str,
    structure: str,
    location: str,
    audience: str,
    company_info: str,
    moment_retriver: VectorStoreRetriever,
    model="gpt_3_5_chat"
): 
    if location == "":
        location = "No specific target location."
    
    if audience == "":
        audience = "No specific target audience. Make it appeal to everyone."
    
    # llm = OpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.5)
    print("using ", model)
    llm = get_llm(model, 0.5)

    moment_query = f"Tell me in detail about {moment}"
    relevant_docs = moment_retriver.get_relevant_documents(moment_query)
    moment_context = "\n".join([item.page_content.replace("\n", " ") for item in relevant_docs])

    # NEW STUFF
    moment_query_template = """Given the following context, i want you to answer this query: {moment_query}

    {moment_context}
    """
    moment_prompt = PromptTemplate(input_variables=["moment_query", "moment_context"], template=moment_query_template)
    moment_chain = LLMChain(llm=get_llm("gpt_3_5_chat"), prompt=moment_prompt, output_key="moment_info")

    # OLD STUFF
    # moment_query_template = "Tell me about {moment_query}. How is it relevant, significant, and important?"
    # moment_prompt = PromptTemplate(input_variables=["moment_query"], template=moment_query_template)
    # moment_chain = LLMChain(llm=llm, prompt=moment_prompt, memory=moment_memory, output_key="moment_info")


    post_template = """Imagine that you are in charge of creating a {content_type}. 

You are to write a {content_type} about {moment_query}. You must relate it with {company_name}.

The content should be targetted towards:
Location: {location}
Target audience: {audience}.

Be creative with the language.

Create a title if a {content_type} requires a title

IMPORTANT INSTRUCTIONS:

Tone of {content_type}: {tone}
Objective of {content_type}: {objective}
Structuring of {content_type}: {structure}

CONTEXT ON {company_name} and {moment_query} follows: 

Information about {company_name}:
{company_info}

Information about {moment_query}:
{moment_info}
"""
    post_prompt = PromptTemplate(input_variables=["company_name", "location", "audience", "moment_query", "company_info", "moment_info", "tone", "objective", "content_type", "structure"], template=post_template)
    post_chain = LLMChain(llm=llm, prompt=post_prompt, output_key="post")

    generator_template = """Given this post text for a {content_type}: {post}

    Tell me what other things can be put in the post. Include description of images, videos, audio, hashtags, etc. as lists, only include elements that are relevant to a {content_type}"""
    generator_prompt = PromptTemplate(input_variables=["post", "content_type"], template=generator_template)
    generator_chain = LLMChain(llm=llm, prompt=generator_prompt, output_key="extras")

    final_chain = SequentialChain(
        chains=[moment_chain, post_chain, generator_chain],
        input_variables=["company_name", "company_info", "location", "audience", "moment_query", "moment_context", "tone", "objective", "content_type", "structure"],
        output_variables=["post", "extras"],
        verbose=True
    )

    return final_chain({
        "company_name": company_name,
        "company_info": company_info,
        "moment_query": moment,
        "moment_context": moment_context,
        "tone": tone,
        "objective": objective,
        "structure": structure,
        "location": location,
        "audience": audience,
        "content_type": content_type
    })


In [4]:
moment = "Looking like a wow trend"
content_category = "Mouth watering food that makes the user crave"
company_name = "Zomato"
content_type = "Instagram"
tone = "casual"
objective = "entertain"
structure = "3 to 4 lines"
location = "Mumbai"
audience = "Zomato gold subscribers"
company_info = """Zomato, a leading global food delivery and restaurant discovery platform, revolutionizes dining experiences. Seamlessly blending technology with gastronomy, it offers a user-friendly app for food enthusiasts. Explore diverse cuisines, read reviews, and conveniently order from local restaurants. Zomato extends beyond delivery, encompassing restaurant reservations, table management, and contactless dining solutions. With keywords like food delivery, restaurant discovery, reviews, reservations, and contactless dining, it caters to diverse culinary needs."""
country_code = "IN"
ref_post = "Mumbai foodies, it's time to upgrade your dining game to a 'WOW' level! With our Zomato Gold subscription, not only will your meals be lip-smacking, but they'll also look Instagram worthy, just like your favorite trend. Let's turn the tables and make every meal a 'Just looking like a wow' moment! #ZomatoGold #WowTrend #Foodie"

In [5]:
moment_context_sitetexts = get_sitetexts(get_related_links(moment.replace("Title: ", ""), country=country_code, num_results=5))
moment_vectorstore, moment_retriver, _, _ = build_vectorstore(moment_context_sitetexts)
moment_memory = VectorStoreRetrieverMemory(
            retriever=moment_retriver,
            input_key="moment_query"
                            )

Getting information on:  Looking like a wow trend 
Getting text: https://indianexpress.com/article/trending/trending-in-india/tmc-mp-nusrat-jahan-joins-deepika-padukone-sanya-malhotra-looking-like-a-wow-trend-9009180/
Getting text: https://www.business-standard.com/entertainment/how-the-just-looking-like-a-wow-trend-is-taking-over-the-internet-123110800746_1.html
Getting text: https://indianexpress.com/article/entertainment/bollywood/deepika-padukones-hilarious-recreation-of-just-looking-like-a-wow-trend-leaves-ranveer-singh-ded-9004204/
Getting text: https://www.hindustantimes.com/entertainment/bollywood/nick-jonas-just-looking-like-a-wow-priyanka-chopra-green-saree-jio-world-plaza-101698906579550.html
Getting text: https://www.socialmediadissect.com/social-media-dissect/so-beautiful-so-elegant-just-looking-like-a-wow-video-goes-viral/
Splitting


Spliting text: 100%|██████████| 5/5 [00:00<00:00, 555.63it/s]

encoding


building retriver


In [6]:
output = generate_similar_content(
    company_name=company_name,
    moment=moment,
    content_type=content_type,
    objective=objective,
    location=location,
    audience=audience,
    company_info=company_info,
    moment_retriver=moment_retriver,
    ref_post=ref_post,
    model="gpt_4_high_temp"
)

c:\Users\vedan\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\llms\openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\vedan\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\llms\openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Reference Post
 ==================== Mumbai foodies, it's time to upgrade your dining game to a 'WOW' level! With our Zomato Gold subscription, not only will your meals be lip-smacking, but they'll also look Instagram worthy, just like your favorite trend. Let's turn the tables and make every meal a 'Just looking like a wow' moment! #ZomatoGold #WowTrend #Foodie


> Entering new SequentialChain chain...


In [ ]:
print(output["post"])

In [5]:
output = generate_content(
    company_name=company_name,
    moment=moment,
    content_type=content_type,
    tone=tone,
    objective=objective,
    structure=structure,
    location=location,
    audience=audience,
    company_info=company_info,
    moment_retriver=moment_retriver,
    model="gpt_4_high_temp"
)

using  gpt_4_high_temp


c:\Users\vedan\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\llms\openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\vedan\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\llms\openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new SequentialChain chain...

> Finished chain.


In [7]:
print(output["post"])

Title: "Dine like a WOW with Zomato Gold!"

"Mumbai foodies, it's time to upgrade your dining game to a 'WOW' level! With our Zomato Gold subscription, not only will your meals be lip-smacking, but they'll also look Instagram worthy, just like your favorite trend. Let's turn the tables and make every meal a 'Just looking like a wow' moment! #ZomatoGold #WowTrend #Foodie"
